## 1D cnn classification based on time course of ROI in brain atlases_ UKBB

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker, NiftiMapsMasker
from nilearn import plotting
from nilearn import image
from nilearn.connectome import ConnectivityMeasure

import nibabel as nib

import warnings

/data/zmohaghegh/venv/lib64/python3.6/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
x=datasets.fetch_atlas_craddock_2012()
img = nib.load('/data/zmohaghegh/nilearn_data/craddock_2012/scorr05_2level_all.nii.gz')
imgdata = img.get_fdata()
imgdata[30,30,34,:]
len(np.unique(imgdata[:,:,:,2]))

31

## get subject info

In [13]:
MDD_UKBB = pd.read_csv('UKBB_2020_MDD_ids.csv')
MDD_UKBB['f.eid'] =MDD_UKBB ['eid']
MDD_UKBB

,eid,mdd,f.eid
0,1003504,1,1003504
1,1003559,1,1003559
2,1015037,1,1015037
3,1034601,1,1034601
4,1054208,1,1054208
...,...,...,...
630,5979530,1,5979530
631,5997732,1,5997732
632,6012549,1,6012549
633,6017679,1,6017679


In [14]:
Control_UKBB= pd.read_csv('control_ids.csv')
Control_UKBB

,f.eid,mdd
0,2224731,0
1,2890106,0
2,2890106,0
3,1699840,0
4,5481734,0
...,...,...
625,5694545,0
626,2730866,0
627,5895433,0
628,5636535,0


## merging subjects info fro MDD and Control

In [16]:
UKBB_merged_subjects = pd.concat([Control_UKBB,MDD_UKBB])
UKBB_merged_subjects

,f.eid,mdd,eid
0,2224731,0,NaN
1,2890106,0,NaN
2,2890106,0,NaN
3,1699840,0,NaN
4,5481734,0,NaN
...,...,...,...
630,5979530,1,5979530.0
631,5997732,1,5997732.0
632,6012549,1,6012549.0
633,6017679,1,6017679.0


In [18]:
# cahnge data_column name = 'SITE_ID':str, 'SUB_ID':int, 'DX_GROUP':int, 'AGE_AT_SCAN':float, 'SEX':str

UKBB_merged_subjects['SUB_ID'] =UKBB_merged_subjects['f.eid']
UKBB_merged_subjects['DX_GROUP'] =UKBB_merged_subjects['mdd']
UKBB_merged_subjects

,f.eid,mdd,eid,SUB_ID,DX_GROUP
0,2224731,0,NaN,2224731,0
1,2890106,0,NaN,2890106,0
2,2890106,0,NaN,2890106,0
3,1699840,0,NaN,1699840,0
4,5481734,0,NaN,5481734,0
...,...,...,...,...,...
630,5979530,1,5979530.0,5979530,1
631,5997732,1,5997732.0,5997732,1
632,6012549,1,6012549.0,6012549,1
633,6017679,1,6017679.0,6017679,1


In [20]:
UKBB_merged_subjects.to_csv('UKBB_merged_subjects_info.csv', index=False)

In [21]:
UKBB_merged_subjects

,f.eid,mdd,eid,SUB_ID,DX_GROUP
0,2224731,0,NaN,2224731,0
1,2890106,0,NaN,2890106,0
2,2890106,0,NaN,2890106,0
3,1699840,0,NaN,1699840,0
4,5481734,0,NaN,5481734,0
...,...,...,...,...,...
630,5979530,1,5979530.0,5979530,1
631,5997732,1,5997732.0,5997732,1
632,6012549,1,6012549.0,6012549,1
633,6017679,1,6017679.0,6017679,1


## Reading subject info after merging

In [109]:
subject_list_info = pd.read_csv('UKBB_merged_subjects_info.csv')

In [110]:
subject_list_info

,f.eid,mdd,eid,SUB_ID,DX_GROUP
0,2224731,0,NaN,2224731,0
1,2890106,0,NaN,2890106,0
2,2890106,0,NaN,2890106,0
3,1699840,0,NaN,1699840,0
4,5481734,0,NaN,5481734,0
...,...,...,...,...,...
1260,5979530,1,5979530.0,5979530,1
1261,5997732,1,5997732.0,5997732,1
1262,6012549,1,6012549.0,6012549,1
1263,6017679,1,6017679.0,6017679,1


In [114]:
list_ids=[]
for ids in subject_list_info['SUB_ID']:
    list_ids.append(f'sub_{ids}')
print(len(list_ids))
subject_list_info['SUB_ID']=list_ids

1265


In [107]:
len(subject_list_info['SUB_ID'])

1265

In [115]:
subject_list_info

,f.eid,mdd,eid,SUB_ID,DX_GROUP
0,2224731,0,NaN,sub_2224731,0
1,2890106,0,NaN,sub_2890106,0
2,2890106,0,NaN,sub_2890106,0
3,1699840,0,NaN,sub_1699840,0
4,5481734,0,NaN,sub_5481734,0
...,...,...,...,...,...
1260,5979530,1,5979530.0,sub_5979530,1
1261,5997732,1,5997732.0,sub_5997732,1
1262,6012549,1,6012549.0,sub_6012549,1
1263,6017679,1,6017679.0,sub_6017679,1


In [116]:
subject_list_info.to_csv('UKBB_merged_subjects_info.csv', index=False)

## Read all subject info

In [11]:
subject_list_info = pd.read_csv('UKBB_merged_subjects_info.csv')
subject_list_info

,f.eid,mdd,eid,SUB_ID,DX_GROUP
0,2224731,0,NaN,sub_2224731,0
1,2890106,0,NaN,sub_2890106,0
2,2890106,0,NaN,sub_2890106,0
3,1699840,0,NaN,sub_1699840,0
4,5481734,0,NaN,sub_5481734,0
...,...,...,...,...,...
1260,5979530,1,5979530.0,sub_5979530,1
1261,5997732,1,5997732.0,sub_5997732,1
1262,6012549,1,6012549.0,sub_6012549,1
1263,6017679,1,6017679.0,sub_6017679,1


In [87]:
available_not= [0,1,2]
rsfmri_bptf_file_list=listofzeros = [0] * 1262
subject_available=subject_list_info.drop(available_not,axis=0)# ['eid']= rsfmri_bptf_file_list
subject_available

,f.eid,mdd,eid,SUB_ID,DX_GROUP
3,1699840,0,NaN,1699840,0
4,5481734,0,NaN,5481734,0
5,3506278,0,NaN,3506278,0
6,3898616,0,NaN,3898616,0
7,5166727,0,NaN,5166727,0
...,...,...,...,...,...
1260,5979530,1,5979530.0,5979530,1
1261,5997732,1,5997732.0,5997732,1
1262,6012549,1,6012549.0,6012549,1
1263,6017679,1,6017679.0,6017679,1


### Create a csv file (Pandas Dataframe) for further machine learning

In [35]:
def create_ml_csv(input_root_dir='/dbstore/zmohaghegh/UKBiobank_subset/New_preprocessed/MDD_Control_2020/',
                  output_root_dir='/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/',
                 time_course_dir = 'tc'):

    subject_list_info = pd.read_csv('UKBB_merged_subjects_info.csv')
    
    # data_column = 'SITE_ID':str, 'SUB_ID':int, 'DX_GROUP':int, 'AGE_AT_SCAN':float, 'SEX':str
    
    # input list
    
    available_not=[]
    rsfmri_bptf_file_list = []
    
    # output list
    tc_file_list = []    # time course filenames
    ccvec_file_list = [] # correlation matrix filenames
    ccmat_file_list = [] # correlation matrix filenames 

    for i,sub_id in enumerate(subject_list_info['f.eid']):
        
        if os.path.exists(f'/dbstore/zmohaghegh/UKBiobank_subset/New_preprocessed/MDD_Control_2020/{sub_id}_20227_2_0/subject_preprocessed.nii.gz'):
            rsfmri_bptf_file = os.path.join(input_root_dir, f'{sub_id}_20227_2_0','subject_preprocessed.nii.gz')         
            rsfmri_bptf_file_list.append(rsfmri_bptf_file)

            #outputs ATLAS-> generic name that is a placeholder
            tc_file_list.append(os.path.join(output_root_dir,f'sub_{sub_id}',time_course_dir,'ATLAS/timecourse.csv'))
            ccvec_file_list.append(os.path.join(output_root_dir, f'sub_{sub_id}',time_course_dir,'ATLAS/corr_vec.npy'))
            ccmat_file_list.append(os.path.join(output_root_dir,f'sub_{sub_id}',time_course_dir,'ATLAS/corr_mat.npy'))
        else:
            available_not.append(i)
            
    # drop out not available subject from data frame        
    subject_available_info = subject_list_info.drop(available_not,axis=0)
    print(f'total number of available subjects= {len(subject_available_info)}')
    
    subject_available_info['RSFMRI_bptf_file'] = rsfmri_bptf_file_list
    subject_available_info['tc_file'] = tc_file_list
    subject_available_info['corrvec_file'] = ccvec_file_list
    subject_available_info['corrmat_file'] = ccmat_file_list
    
    return subject_available_info

### Choose one of the atlases 

In [28]:
def select_atlas(atlas_name, atlas_dir='/data/zmohaghegh/nilearn_data/'): 

    # Choose one of the atlases (add more when necessary)
    # 1. AAL
    # 2. HO_cort_maxprob_thr25-2mm


    # Check if valid atlas name
    if atlas_name not in ['AAL', 'HO_cort_maxprob_thr25-2mm', 'schaefer_100', 'schaefer_400',
                         'JAMA_IC19', 'JAMA_IC52', 'JAMA_IC7']:
        raise ValueError('atlas_name not found')

    if atlas_name == 'AAL':
        dataset = datasets.fetch_atlas_aal(version='SPM12')
        atlas_filename = dataset.maps
        labels = dataset.labels
        
    if atlas_name == 'HO_cort_maxprob_thr25-2mm':
        dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
        atlas_filename = dataset.maps
        labels = dataset.labels[1:] # the first element is background
        
    if atlas_name == 'schaefer_100':
        atlas_filename = os.path.join(atlas_dir,'schaefer/Schaefer2018_100Parcels_17Networks_order_FSLMNI152_2mm.nii')
        labels = pd.read_csv(os.path.join(atlas_dir, 
                                           'schaefer/Schaefer2018_100Parcels_17Networks_table.csv'))['label']
    if atlas_name == 'schaefer_400':
        atlas_filename = os.path.join(atlas_dir, 
                                       'schaefer/Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2mm.nii')
        labels = pd.read_csv(os.path.join(atlas_dir, 
                                           'schaefer/Schaefer2018_400Parcels_17Networks_table.csv'))['label']
        
    if atlas_name == 'JAMA_IC19':
        atlas_filename = os.path.join(atlas_dir, 'IC_JAMA/IC19.nii')
        labels =[f'JAMA_IC19_{i+1}' for i in range(19)]
    
    if atlas_name == 'JAMA_IC52':
        atlas_filename = os.path.join(atlas_dir, 'IC_JAMA/IC52.nii')
        labels =[f'JAMA_IC52_{i+1}' for i in range(52)]
        
    if atlas_name == 'JAMA_IC7':
        atlas_filename = os.path.join(atlas_dir, 'IC_JAMA/IC7.nii')
        labels =[f'JAMA_IC7_{i+1}' for i in range(7)]
        
    return  atlas_filename, labels, len(labels)
    
    #plotting.plot_roi(atlas_filename)

### Masking

In [29]:
def label_masker(atlas_filename):
    masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True)     
    return masker

### Extract time course

In [30]:
def extract_tc_corr(atlas_names,
                    output_root_dir='/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/',
                   time_course_dir = 'tc'):

    df_data_info= create_ml_csv()
    #print(len(df_data_info))
    
    # Write the generic input and output csv files
    
    df_data_info.to_csv(os.path.join(output_root_dir, 'data_info.csv'), index=False)

    connectivity_measure = ConnectivityMeasure(kind='correlation')
    
    nsubjects = len(df_data_info)
    
    print_counter = 0
    
    for i,sub_i in enumerate(df_data_info.index): #sub_i in df_data_info.index
        #print(sub_i)
        
        if print_counter%100 == 0:
            print(f'{i}/{nsubjects}')

        for atlas_name in atlas_names:
            
            atlas_filename, labels, nrois = select_atlas(atlas_name)
            
            if atlas_name in ['JAMA_IC19', 'JAMA_IC52', 'JAMA_IC7']:
                masker = NiftiMapsMasker(atlas_filename,  standardize=True, memory='nilearn_cache')
                
            else:
                masker = label_masker(atlas_filename)

            atlas_bptf_conf_name = os.path.join(atlas_name)
            timecourse = masker.fit_transform(imgs=df_data_info['RSFMRI_bptf_file'].loc[sub_i])  
    
            cc = connectivity_measure.fit_transform([timecourse])[0]

            tc_dir = os.path.join(output_root_dir, df_data_info['SUB_ID'].loc[sub_i],
                                  time_course_dir, atlas_bptf_conf_name)
            
            if not os.path.exists(tc_dir):
                os.makedirs(tc_dir)

            # Write timeseries as csv file
            tc_file = df_data_info['tc_file'].loc[sub_i].replace('ATLAS/', atlas_bptf_conf_name)
            pd.DataFrame(data=timecourse, columns=labels).to_csv(tc_file, index=False)

            # Write correlation matrix
            corrvec_file = df_data_info['corrvec_file'].loc[sub_i].replace('ATLAS/', atlas_bptf_conf_name)
            corrmat_file = df_data_info['corrmat_file'].loc[sub_i].replace('ATLAS/', atlas_bptf_conf_name)
            cc_triu_ids = np.triu_indices(nrois)
            
            np.save(corrvec_file, cc[cc_triu_ids]) # get only upper triangular
            np.save(corrmat_file, cc)
            
        print_counter += 1

    return None

In [42]:
df_data_info= create_ml_csv()
df_data_info

number of subject=818


,f.eid,mdd,eid,SUB_ID,DX_GROUP,RSFMRI_bptf_file,tc_file,corrvec_file,corrmat_file
1,2890106,0,NaN,sub_2890106,0,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...
2,2890106,0,NaN,sub_2890106,0,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...
3,1699840,0,NaN,sub_1699840,0,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...
5,3506278,0,NaN,sub_3506278,0,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...
7,5166727,0,NaN,sub_5166727,0,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...
...,...,...,...,...,...,...,...,...,...
1257,5969480,1,5969480.0,sub_5969480,1,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...
1258,5971173,1,5971173.0,sub_5971173,1,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...
1259,5977819,1,5977819.0,sub_5977819,1,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...
1260,5979530,1,5979530.0,sub_5979530,1,/dbstore/zmohaghegh/UKBiobank_subset/New_prepr...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...,/dbstore/zmohaghegh/UKBiobank_subset/Time_cour...


### Running Extractio of time course

In [36]:
atlas_names=['AAL', 'HO_cort_maxprob_thr25-2mm'] # 'schaefer_100', 'schaefer_400'

In [37]:
extract_tc_corr(atlas_names)

number of subject=818
818
1/818
158/818
290/818
433/818
604/818
761/818
914/818
1074/818
1236/818


In [39]:
import shutil

for file in os.listdir('/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/'):
    print(file)
    shutil.move(f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/AALtimecourse.csv', 
                f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/AAL/timecourse.csv')
    shutil.move(f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/AALcorr_mat.npy', 
                f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/AAL/corr_mat.npy')
    shutil.move(f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/AALcorr_vec.npy', 
                f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/AAL/corr_vec.npy')

sub_2890106
sub_1699840
sub_3506278
sub_5166727
sub_2385770
sub_3686904
sub_1343733
sub_3210483
sub_5405306
sub_4424127
sub_5237704
sub_3795465
sub_5515158
sub_1261334
sub_1119288
sub_4583789
sub_3852329
sub_3398650
sub_4616229
sub_5260559
sub_3881427
sub_2818054
sub_5065521
sub_4483688
sub_1856492
sub_4418666
sub_2577430
sub_5228667
sub_1506586
sub_1190429
sub_4497866
sub_3216289
sub_4711331
sub_5575539
sub_4904501
sub_2817815
sub_5037979
sub_1793935
sub_3506600
sub_3248642
sub_5748826
sub_3339838
sub_2233347
sub_3049661
sub_1117296
sub_4655739
sub_4581302
sub_1866080
sub_4027737
sub_2205549
sub_2099413
sub_5200537
sub_1372955
sub_2957890
sub_1627544
sub_5506100
sub_1698918
sub_3165520
sub_5550357
sub_2740061
sub_1099649
sub_2592764
sub_4280370
sub_1051042
sub_2475689
sub_4458765
sub_2297860
sub_4175006
sub_1374303
sub_5188778
sub_2339635
sub_4020928
sub_2505998
sub_3512260
sub_2892204
sub_5797137
sub_5089573
sub_3672549
sub_3463734
sub_2685701
sub_5101024
sub_5963913
sub_1852761
sub_

sub_4352219
sub_4357535
sub_4362680
sub_4394444
sub_4399626
sub_4404436
sub_4438596
sub_4442378
sub_4451675
sub_4484371
sub_4487230
sub_4518055
sub_4526895
sub_4527897
sub_4529423
sub_4556036
sub_4564994
sub_4567733
sub_4567759
sub_4576247
sub_4584432
sub_4584879
sub_4585027
sub_4611810
sub_4611852
sub_4612723
sub_4625238
sub_4626289
sub_4641584
sub_4643848
sub_4647465
sub_4648981
sub_4651835
sub_4661801
sub_4672867
sub_4681094
sub_4693741
sub_4723273
sub_4729554
sub_4761760
sub_4770271
sub_4785929
sub_4795791
sub_4802825
sub_4805131
sub_4808863
sub_4827195
sub_4844441
sub_4853612
sub_4854899
sub_4857193
sub_4875866
sub_4904831
sub_4906319
sub_4910592
sub_4916197
sub_4919104
sub_4921548
sub_4967185
sub_4969835
sub_5008876
sub_5010065
sub_5011478
sub_5014585
sub_5077228
sub_5092190
sub_5117007
sub_5123303
sub_5130208
sub_5172700
sub_5174797
sub_5178366
sub_5199356
sub_5215202
sub_5215721
sub_5227706
sub_5258214
sub_5274624
sub_5283773
sub_5290220
sub_5345237
sub_5396473
sub_5416588
sub_

In [40]:
for file in os.listdir('/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/'):
    if os.path.exists(f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/HO_cort_maxprob_thr25-2mmtimecourse.csv'):
        print(file)
        shutil.move(f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/HO_cort_maxprob_thr25-2mmtimecourse.csv', 
                    f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/HO_cort_maxprob_thr25-2mm/timecourse.csv')
        shutil.move(f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/HO_cort_maxprob_thr25-2mmcorr_mat.npy', 
                    f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/HO_cort_maxprob_thr25-2mm/corr_mat.npy')
        shutil.move(f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/HO_cort_maxprob_thr25-2mmcorr_vec.npy', 
                    f'/dbstore/zmohaghegh/UKBiobank_subset/Time_course_New/{file}/tc/HO_cort_maxprob_thr25-2mm/corr_vec.npy')

sub_2890106
sub_1699840
sub_3506278
sub_5166727
sub_2385770
sub_3686904
sub_1343733
sub_3210483
sub_5405306
sub_4424127
sub_5237704
sub_3795465
sub_5515158
sub_1261334
sub_1119288
sub_4583789
sub_3852329
sub_3398650
sub_4616229
sub_5260559
sub_3881427
sub_2818054
sub_5065521
sub_4483688
sub_1856492
sub_4418666
sub_2577430
sub_5228667
sub_1506586
sub_1190429
sub_4497866
sub_3216289
sub_4711331
sub_5575539
sub_4904501
sub_2817815
sub_5037979
sub_1793935
sub_3506600
sub_3248642
sub_5748826
sub_3339838
sub_2233347
sub_3049661
sub_1117296
sub_4655739
sub_4581302
sub_1866080
sub_4027737
sub_2205549
sub_2099413
sub_5200537
sub_1372955
sub_2957890
sub_1627544
sub_5506100
sub_1698918
sub_3165520
sub_5550357
sub_2740061
sub_1099649
sub_2592764
sub_4280370
sub_1051042
sub_2475689
sub_4458765
sub_2297860
sub_4175006
sub_1374303
sub_5188778
sub_2339635
sub_4020928
sub_2505998
sub_3512260
sub_2892204
sub_5797137
sub_5089573
sub_3672549
sub_3463734
sub_2685701
sub_5101024
sub_5963913
sub_1852761
sub_

sub_4349094
sub_4352219
sub_4357535
sub_4362680
sub_4394444
sub_4399626
sub_4404436
sub_4438596
sub_4442378
sub_4451675
sub_4484371
sub_4487230
sub_4518055
sub_4526895
sub_4527897
sub_4529423
sub_4556036
sub_4564994
sub_4567733
sub_4567759
sub_4576247
sub_4584432
sub_4584879
sub_4585027
sub_4611810
sub_4611852
sub_4612723
sub_4625238
sub_4626289
sub_4641584
sub_4643848
sub_4647465
sub_4648981
sub_4651835
sub_4661801
sub_4672867
sub_4681094
sub_4693741
sub_4723273
sub_4729554
sub_4761760
sub_4770271
sub_4785929
sub_4795791
sub_4802825
sub_4805131
sub_4808863
sub_4827195
sub_4844441
sub_4853612
sub_4854899
sub_4857193
sub_4875866
sub_4904831
sub_4906319
sub_4910592
sub_4916197
sub_4919104
sub_4921548
sub_4967185
sub_4969835
sub_5008876
sub_5010065
sub_5011478
sub_5014585
sub_5077228
sub_5092190
sub_5117007
sub_5123303
sub_5130208
sub_5172700
sub_5174797
sub_5178366
sub_5199356
sub_5215202
sub_5215721
sub_5227706
sub_5258214
sub_5274624
sub_5283773
sub_5290220
sub_5345237
sub_5396473
sub_

## Data Exploration

In [2]:
time_course_sub = pd.read_csv('/dbstore/zmohaghegh/UKBiobank_subset/Time_course/sub_1119288/tc/AAL/timecourse.csv')
time_course_sub

,Precentral_L,Precentral_R,Frontal_Sup_L,Frontal_Sup_R,Frontal_Sup_Orb_L,Frontal_Sup_Orb_R,Frontal_Mid_L,Frontal_Mid_R,Frontal_Mid_Orb_L,Frontal_Mid_Orb_R,...,Cerebelum_10_L,Cerebelum_10_R,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9,Vermis_10
0,1.318614,3.012848,1.186333,1.316328,-0.944385,-0.005034,-0.374944,0.758948,0.262717,1.118455,...,0.071786,0.309743,-3.922010,-0.464389,2.399318,0.859113,0.075110,-0.881551,-0.286059,0.291017
1,1.180839,2.742517,0.993617,1.203716,-0.966537,0.032453,-0.428017,0.709821,0.124409,0.985181,...,0.058707,0.264676,-3.763458,-0.469316,2.155850,0.686347,-0.012943,-0.873044,-0.281345,0.159053
2,1.046849,2.473924,0.806597,1.095817,-0.984647,0.068958,-0.476338,0.665233,-0.013463,0.851058,...,0.048706,0.228036,-3.588039,-0.460448,1.920995,0.517552,-0.087208,-0.852841,-0.274745,0.030066
3,0.918092,2.213285,0.630703,0.992414,-0.993217,0.104154,-0.516718,0.623345,-0.148608,0.719579,...,0.045629,0.197624,-3.409246,-0.441726,1.692231,0.361864,-0.160220,-0.834232,-0.264373,-0.090983
4,0.797741,1.963067,0.464212,0.893240,-0.999038,0.129527,-0.550325,0.585767,-0.279390,0.590556,...,0.046014,0.184800,-3.220332,-0.409210,1.475652,0.214317,-0.222265,-0.800736,-0.247715,-0.196158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,-0.355087,-0.455689,0.991361,0.556346,0.475777,-0.731210,1.060028,0.474695,-0.014445,-0.170169,...,1.184208,0.723409,0.981300,0.042080,0.664372,0.945099,0.797396,-0.687488,0.415440,2.360765
519,-0.332570,-0.498656,0.984060,0.544430,0.445701,-0.761331,1.091318,0.489400,0.011842,-0.191232,...,1.227674,0.722310,0.994794,0.079523,0.688950,0.986603,0.862260,-0.696527,0.428641,2.357789
520,-0.315091,-0.545829,0.968844,0.529323,0.414818,-0.790960,1.113656,0.497632,0.035512,-0.211823,...,1.257677,0.714982,1.011661,0.114996,0.713528,1.023539,0.915531,-0.701312,0.435555,2.339929
521,-0.302872,-0.595242,0.946442,0.511050,0.382479,-0.824355,1.128826,0.500055,0.057763,-0.236380,...,1.284218,0.694097,1.020095,0.147512,0.725922,1.057894,0.964101,-0.711414,0.440898,2.308178


In [5]:
correlation_vector=np.load('/dbstore/zmohaghegh/UKBiobank_subset/Time_course/sub_1119288/tc/AAL/corr_vec.npy')
correlation_vector

array([1.        , 0.51860051, 0.13913495, ..., 1.        , 0.5878738 ,
       1.        ])

In [6]:
len(correlation_vector)

6786

In [8]:
correlation_matrix=np.load('/dbstore/zmohaghegh/UKBiobank_subset/Time_course/sub_1119288/tc/AAL/corr_mat.npy')
correlation_matrix

array([[ 1.        ,  0.51860051,  0.13913495, ...,  0.19170488,
         0.31726741, -0.11325635],
       [ 0.51860051,  1.        ,  0.19318602, ...,  0.46328394,
         0.64051391,  0.14290268],
       [ 0.13913495,  0.19318602,  1.        , ...,  0.4302881 ,
         0.2870508 ,  0.4445648 ],
       ...,
       [ 0.19170488,  0.46328394,  0.4302881 , ...,  1.        ,
         0.58387959,  0.24129146],
       [ 0.31726741,  0.64051391,  0.2870508 , ...,  0.58387959,
         1.        ,  0.5878738 ],
       [-0.11325635,  0.14290268,  0.4445648 , ...,  0.24129146,
         0.5878738 ,  1.        ]])

In [9]:
len(correlation_matrix)

116

In [132]:
xx=datasets.fetch_atlas_aal()
len(xx.labels)

116

In [133]:
xx1=datasets.fetch_atlas_harvard_oxford(atlas_name='cort-maxprob-thr25-2mm')
len(xx1.labels)

49

In [134]:
xx

{'description': b'AAl atlas for SPM 12\n\n\nNotes\n-----\nThis atlas is the result of an automated anatomical parcellation of the spatially normalized single-subject high-resolution T1 volume provided by the Montreal Neurological Institute (MNI) (D. L. Collins et al., 1998, Trans. Med. Imag. 17, 463-468, PubMed).\n\nUsing this parcellation method, three procedures to perform the automated anatomical labeling of functional studies are proposed: (1) labeling of an extremum defined by a set of coordinates, (2) percentage of voxels belonging to each of the AVOI intersected by a sphere centered by a set of coordinates, and (3) percentage of voxels belonging to each of the AVOI intersected by an activated cluster.\n\n\nContent\n-------\n    :"regions": str. path to nifti file containing regions.\n    :"labels": dict. labels dictionary with their region id as key and name as value\n\n\nReferences\n-----\nFor more information on this dataset\'s structure, see\nhttp://www.gin.cnrs.fr/AAL-217?la

In [135]:
xx1

{'maps': '/data/zmohaghegh/nilearn_data/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr25-2mm.nii.gz',
 'labels': ['Background',
  'Frontal Pole',
  'Insular Cortex',
  'Superior Frontal Gyrus',
  'Middle Frontal Gyrus',
  'Inferior Frontal Gyrus, pars triangularis',
  'Inferior Frontal Gyrus, pars opercularis',
  'Precentral Gyrus',
  'Temporal Pole',
  'Superior Temporal Gyrus, anterior division',
  'Superior Temporal Gyrus, posterior division',
  'Middle Temporal Gyrus, anterior division',
  'Middle Temporal Gyrus, posterior division',
  'Middle Temporal Gyrus, temporooccipital part',
  'Inferior Temporal Gyrus, anterior division',
  'Inferior Temporal Gyrus, posterior division',
  'Inferior Temporal Gyrus, temporooccipital part',
  'Postcentral Gyrus',
  'Superior Parietal Lobule',
  'Supramarginal Gyrus, anterior division',
  'Supramarginal Gyrus, posterior division',
  'Angular Gyrus',
  'Lateral Occipital Cortex, superior division',
  'Lateral Occipital Cortex, inf